In [1]:
import os
import logging
import shutil

from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.utilities import SerpAPIWrapper
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder
import gradio as gr

# T6

In [ ]:
# --- Configuration ---
LLM_MODEL = "llama3"  
EMBEDDING_MODEL = "nomic-embed-text" 
DOCS_DIR = "langchain_docs" 
VECTOR_STORE_PATH = "vector_store_faiss"
SERPAPI_API_KEY = os.getenv("2bed125e54b6e98cd01e12bbaf2865fc76f707b499ff03bd70cd1536d62cb0af")  

# --- Initialize LLM and Embeddings ---
try:
    llm = Ollama(model=LLM_MODEL, temperature=0.1) # varity in Answer
    logging.info(f"Ollama LLM ({LLM_MODEL}) initialized with temperature 0.1.")
    embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)
    logging.info(f"Ollama Embeddings ({EMBEDDING_MODEL}) initialized.")
except Exception as e:
    logging.error(f"Error initializing Ollama. Is Ollama running and the model '{LLM_MODEL}'/'{EMBEDDING_MODEL}' pulled? Error: {e}")
    llm = None
    embeddings = None


# --- Document Loading and Vector Store Setup ---
def setup_vector_store(force_recreate=False):
    if not embeddings:
        logging.error("Embeddings not initialized. Cannot setup vector store.")
        return None

    if os.path.exists(VECTOR_STORE_PATH) and not force_recreate:
        try:
            logging.info(f"Loading existing vector store from {VECTOR_STORE_PATH}")
            db = FAISS.load_local(VECTOR_STORE_PATH, embeddings, allow_dangerous_deserialization=True)
            logging.info("Vector store loaded successfully.")
            return db
        except Exception as e:
            logging.warning(f"Failed to load existing vector store: {e}. Recreating...")
            if os.path.exists(VECTOR_STORE_PATH):
                shutil.rmtree(VECTOR_STORE_PATH)

    logging.info(f"Creating new vector store from documents in {DOCS_DIR}")
    if not os.path.exists(DOCS_DIR) or not os.listdir(DOCS_DIR):
        logging.error(f"Docs directory '{DOCS_DIR}' is empty or does not exist. Please create it and add markdown files.")
        return None

    try:
        loader = DirectoryLoader(
            DOCS_DIR,
            glob="**/*.md",
            loader_cls=TextLoader,
            show_progress=True,
            use_multithreading=True,
            loader_kwargs={'encoding': 'utf-8'}
        )
        documents = loader.load()

        if not documents:
            logging.error("No documents loaded from the directory. Cannot create vector store.")
            return None
        logging.info(f"Loaded {len(documents)} documents.")

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        texts = text_splitter.split_documents(documents)

        # --- Limit to 50 chunks only ---
        texts = texts[:50]
        logging.info(f"Trimmed to {len(texts)} chunks for vector store.")

        db = FAISS.fr6om_documents(texts, embeddings)
        db.save_local(VECTOR_STORE_PATH)
        logging.info(f"Vector store created and saved to {VECTOR_STORE_PATH}")
        return db
    except Exception as e:
        logging.error(f"Error creating vector store: {e}", exc_info=True)
        return None

C:\Users\menna\AppData\Local\Temp\ipykernel_33352\258570342.py:9: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model=LLM_MODEL, temperature=0.1) # Added temperature
C:\Users\menna\AppData\Local\Temp\ipykernel_33352\258570342.py:11: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)


In [3]:
# --- Run Setup ---
vector_store_db = setup_vector_store()
retriever = None
if vector_store_db:
    retriever = vector_store_db.as_retriever(search_kwargs={"k": 3})
    logging.info("Retriever created from vector store.")
else:
    logging.warning("Vector store setup failed. RAG tool will not be available.")

In [4]:
import os

# Set the key temporarily in the notebook session
os.environ["SERPAPI_API_KEY"] = "2bed125e54b6e98cd01e12bbaf2865fc76f707b499ff03bd70cd1536d62cb0af"


In [5]:
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY", "2bed125e54b6e98cd01e12bbaf2865fc76f707b499ff03bd70cd1536d62cb0af")


In [6]:
print("Key:", SERPAPI_API_KEY)  # Should now show your key


Key: 2bed125e54b6e98cd01e12bbaf2865fc76f707b499ff03bd70cd1536d62cb0af


In [7]:
# Set the SerpAPI key
# IMPORTANT: Replace "YOUR_SERPAPI_API_KEY" with your actual key or ensure it's set as an environment variable
os.environ["SERPAPI_API_KEY"] = "2bed125e54b6e98cd01e12bbaf2865fc76f707b499ff03bd70cd1536d62cb0af" # Replace with your key if hardcoding
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")

if SERPAPI_API_KEY:
    print(f"SERPAPI_API_KEY loaded: {SERPAPI_API_KEY[:4]}...{SERPAPI_API_KEY[-4:]}") # Print a redacted version
else:
    print("SERPAPI_API_KEY not found. Search tool will be disabled.")

SERPAPI_API_KEY loaded: 2bed...b0af


In [8]:
tools = []

from langchain.tools import Tool

# 1. RAG Tool (LangChain Documentation)
if retriever:
    def langchain_doc_search_tool(query: str) -> str:
        # Perform similarity search using your vectorstore
        docs = vector_store_db.similarity_search(query, k=5)

        return "\n\n".join([doc.page_content for doc in docs])

    rag_tool = Tool(
        name="langchain_documentation_search",
        func=langchain_doc_search_tool,
        description="Searches and returns information from the LangChain Python documentation. Use this for questions about LangChain concepts, how-tos, modules, classes, functions, or examples."
    )

    tools.append(rag_tool)

# 2. Search Tool (SerpAPI)
if SERPAPI_API_KEY:
    try:
        search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
        # Test query (optional, but good for immediate feedback)
        # search.run("test query")
        logging.info("SerpAPIWrapper initialized successfully.")
        tools.append(Tool(
            name="internet_search",
            func=search.run,
            description="A search engine. Useful for when you need to answer questions about current events, general knowledge, or topics not covered by the LangChain documentation. Input should be a search query."
        ))
    except Exception as e:
        logging.error(f"SerpAPI initialization or test failed: {e}. Search tool will not be available.")
else:
    logging.warning("SERPAPI_API_KEY not found. Internet search tool will not be available.")

# --- Updated Python REPL Tool Configuration ---
python_repl_tool = PythonREPLTool()
tools.append(Tool(
    name="python_repl",
    func=lambda code: f"```python\n{code}\n```\nResult:\n{python_repl_tool.run(code)}",
    description="A Python REPL. Use this to execute Python code to answer questions, perform calculations, or dynamically reason. Input should be valid Python code.",
    return_direct=True
))
logging.info(f"Available tools: {[tool.name for tool in tools]}")

In [9]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chat_history_placeholder = MessagesPlaceholder(variable_name="chat_history")

# --- Initialize Agent ---
agent_executor = None
if llm and tools:
    try:
        agent_executor = initialize_agent(
            tools,
            llm,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            verbose=True,
            memory=memory,
            agent_kwargs={
                "extra_prompt_messages": [chat_history_placeholder],
                "max_iterations": 15,
                "early_stopping_method": "generate"
            },
            handle_parsing_errors=True,
            max_execution_time=60
        )
        logging.info("LangChain Agent (CHAT_CONVERSATIONAL_REACT_DESCRIPTION) initialized.")
    except Exception as e:
        logging.error(f"Error initializing agent: {e}", exc_info=True)
else:
    logging.error("LLM or tools not available. Agent cannot be initialized.")

C:\Users\menna\AppData\Local\Temp\ipykernel_33352\1728227744.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
C:\Users\menna\AppData\Local\Temp\ipykernel_33352\1728227744.py:8: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor

In [10]:
# --- Gradio Chat Function ---
def chat_with_agent(message, history):
    if not agent_executor:
        return "Agent not initialized. Please check Ollama setup and logs."
    try:
        logging.info(f"User Query: {message}")
        # The agent_executor.invoke now directly takes the input dictionary
        response = agent_executor.invoke({
            "input": message
            # chat_history is handled by the memory object passed to the agent
        })

        bot_message = response.get("output", "")

        # Handle cases where the agent might stop prematurely or have parsing issues
        if "Agent stopped due to iteration limit" in bot_message or "Agent stopped due to time limit" in bot_message:
            return "I was taking too long to think about this. Here's what I have so far:\n\n" + bot_message.split("Thought:")[-1]

        if "Could not parse LLM output" in bot_message:
            if "Final Answer" in bot_message: # Attempt to extract Final Answer
                try:
                    # More robust parsing for Final Answer
                    action_input_start = bot_message.rfind('"action_input":')
                    if action_input_start != -1:
                        json_like_part = bot_message[action_input_start:]
                        # Find the corresponding closing brace for the action_input value
                        # This is a simplified parser, assumes action_input is a string
                        val_start = json_like_part.find(':') + 1
                        val_stripped = json_like_part[val_start:].lstrip().strip('"')
                        # Find the end of the string value for action_input
                        # This might not be perfect for all escaped strings
                        end_quote_idx = val_stripped.find('"')
                        if end_quote_idx != -1:
                             return val_stripped[:end_quote_idx]

                except Exception:
                    pass # Fallback to simpler extraction
            return "I encountered a formatting issue but here's my response:\n\n" + bot_message.split("Thought:")[-1]


        return bot_message if bot_message else "Sorry, I could not process that."

    except Exception as e:
        error_msg = str(e)
        if "iteration limit" in error_msg.lower() or "time limit" in error_msg.lower():
            return "I was taking too long to think about this. Please try asking a more specific question."
        logging.error(f"Error during agent execution: {e}", exc_info=True)
        return f"Sorry, an error occurred: {error_msg}"

In [ ]:
import gradio as gr
# Assuming LLM_MODEL, EMBEDDING_MODEL, tools, chat_with_agent, llm, embeddings, logging
# are defined in previous cells/parts of your script.

# --- Gradio Interface ---
iface = gr.ChatInterface(
    fn=chat_with_agent,
    title="Conversational RAG with LangChain & Ollama 🚀",
    description="Ask technical questions about LangChain, current events, or request Python code execution. \n"
                f"Using LLM: {LLM_MODEL}, Embeddings: {EMBEDDING_MODEL}. \n"
                f"Tools: {[tool.name for tool in tools] if tools else 'None available'}. \n" # Changed 'None' to 'None available' for clarity
                "Ensure Ollama is running and the models are pulled (e.g., `ollama pull llama3`).",
    examples=[
        "How do I build a custom agent in LangChain?",
        "What are the key features of llama3 according to recent news?",
        "What's 2 to the power of 20 in Python?",
        "Summarize the LangChain Expression Language (LCEL)."
    ],
    cache_examples=False # Added cache_examples=False
)


# --- Main Execution ---
if __name__ == "__main__":

    if not llm or not embeddings:
        logging.critical("LLM or Embeddings could not be initialized. Application cannot start.")
        print("CRITICAL: LLM or Embeddings could not be initialized. Application cannot start.") # Added print for immediate visibility
        print("Please ensure Ollama is running and the specified models are pulled.")
        print(f"Attempted LLM: {LLM_MODEL}, Attempted Embeddings: {EMBEDDING_MODEL}")
    # Added a check for agent_executor as well, as it's crucial
    elif 'agent_executor' not in globals() or not agent_executor:
        logging.critical("Agent Executor could not be initialized. Application cannot start.")
        print("CRITICAL: Agent Executor could not be initialized. Check tool setup and LLM availability.")
    else:
        logging.info("Launching Gradio interface from __main__ block...") # Added logging
        print("Launching Gradio interface...") # Added print for immediate visibility
        iface.launch(share=False) # Set share=True to create a public link (requires ngrok or similar)

c:\Users\menna\miniconda3\envs\newtorch\Lib\site-packages\gradio\chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Launching Gradio interface...
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...


```json
{
    "action": "python_repl",
    "action_input": "2 ** 20"
}
```

This will execute the Python code `2 ** 20` in a REPL, which will calculate and return the result.
Observation: ```python
2 ** 20
```
Result:



> Finished chain.


> Entering new AgentExecutor chain...
```json
{
    "action": "python_repl",
    "action_input": "print(2 ** 20)"
}
```

This will execute the Python code `print(2 ** 20)` and return the result, which is the answer to the original question.
Observation: ```python
print(2 ** 20)
```
Result:
1048576



> Finished chain.


> Entering new AgentExecutor chain...
```json
{
    "action": "langchain_documentation_search",
    "action_input": "custom agents langchain"
}
```

This will search the LangChain documentation for information on building custom agents.
Observation: Please see the following resources for more information:

  * LangGraph docs on [common agent architectures](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/)
  * [Pre-